In [19]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as f 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 

# Set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set Hyperparameters 
input_size = 28 
sequenc_length = 28 
num_layers = 2 
hidden_size = 256 
num_classes = 10 
learning_rate = 0.001 
batch_size = 64 
mun_epochs = 2 

In [29]:
class BRNN(nn.Module): 
    def __init__(self, input_zie, hidden_size, num_layers, num_classes): 
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size 
        self.num_layers = num_layers 
        self.lstm = nn.LSTM(input_zie, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        
    def forward(self, x): 
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

In [30]:
# Initialize Network
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Lass and Optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
# Load data 
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [ ]:

# Train the Network 
for epoch in range(num_epochs): 
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        # forward 
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient desent or adam step
        optimizer.step()

        
        
# Check accuracy 
def check_accuracy(loader, model): 
    if loader.dataset.train: 
        print("Checking accuracy on training data...")
    else:
        print("Checking accuracy on test data...")
        
    num_correct = 0 
    num_samples = 0 
    model.eval() 
    
    with torch.no_grad(): 
        for x, y in loader: 
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
            
            scores = model(x)  # RNN()
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
        print(f'got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}')
        
    model.train()
